In [1]:
import re
from collections import Counter

with open('var10.txt', 'r', encoding='utf-8') as f:
    cipher = re.sub(r'\n', '', f.read())

alphabet = "абвгдежзийклмнопрстуфхцчшщьыэюя" 
letter_to_index = {ch:i for i,ch in enumerate(alphabet)}
index_to_letter = {i:ch for i,ch in enumerate(alphabet)}

# +-------------------------------------------------------------+
def gcdextended(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = gcdextended(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def modinv(a, m):
    gcd, x, y = gcdextended(a, m)
    if gcd != 1:
        return None   
    return (x % m + m) % m

def liner(a, b, m):
    gcd, x0, _ = gcdextended(a, m)
    if b % gcd != 0:
        return []  
    a1 = a // gcd
    b1 = b // gcd
    m1 = m // gcd
    x0 = (b1 * modinv(a1, m1)) % m1
    sols = [(x0 + k * m1) % m for k in range(gcd)]
    return sols

def bigram_to_num(bg):
    return 31*letter_to_index[bg[0]] + letter_to_index[bg[1]]   
# +-------------------------------------------------------------+


bigrams = [cipher[i:i+2] for i in range(0, len(cipher), 2)]
freq = Counter(bigrams)

top5_cipher = []
for bg, count in freq.most_common(5):
    top5_cipher.append(bg)
print(top5_cipher)

top5_plain = ["ст","но","то","на","ен"]

plaintext_vals = {}
for bg in top5_plain:
    plaintext_vals[bg] = bigram_to_num(bg)       

cipher_vals = {}
for bg in top5_cipher:
    cipher_vals[bg] = bigram_to_num(bg)      


best_count = -1
best_key = None
best_plain = ""

for p1 in top5_plain:
    for p2 in top5_plain:
        if p2 == p1: 
            continue
        X1 = plaintext_vals[p1]
        X2 = plaintext_vals[p2]
        for c1 in top5_cipher:
            for c2 in top5_cipher:
                if c2 == c1:
                    continue
                Y1 = cipher_vals[c1]
                Y2 = cipher_vals[c2]


                A = (X1 - X2) % 961
                B = (Y1 - Y2) % 961
                sol_a = liner(A, B, 961)
                for a in sol_a:
                    if gcdextended(a, 961)[0] != 1:
                        continue
                    b = (Y1 - a*X1) % 961
                    a_inv = modinv(a, 961)


                    plaintext = []
                    for bg in bigrams:
                        Y = bigram_to_num(bg)
                        X = (a_inv * ((Y - b) % 961)) % 961
                        i = X // 31
                        j = X % 31
                        plaintext.append(index_to_letter[i])
                        plaintext.append(index_to_letter[j])
                    text = "".join(plaintext)

                    cnt = 0
                    targets = set(top5_plain)
                    for i in range(len(text)-1):
                        if text[i:i+2] in targets:
                            cnt += 1
                    if cnt > best_count:
                        best_count = cnt
                        best_plain = text
                        best_key = (a, b)

with open('decrypted.txt', 'w', encoding='utf-8') as f:
    f.write(best_plain)

print(f"Best key: a={best_key[0]}, b={best_key[1]}")
print(f"Decrypted plaintext:\n{best_plain}")

['сг', 'жэ', 'ям', 'нг', 'тм']
Best key: a=300, b=400
Decrypted plaintext:
поздновечеромнаверандесиделколяичтотописалвтемнотебумагуитутолкомнельзябылоразглядетьвремяотвременионвосклицалагаилииэтотожезначитемувголовуприходилоещечтонибудьподходящеедляегоспискапотомдверьчутьстукнулаточновсеткуотмоскитовудариласьночнаябабочкалинашепнулауфманонаселарядомснимнакачеливоднойночнойсорочкенетоненькаякаксемнадцатилетняядевочкакоторуюещенелюбятинетолстаякакпятидесятилетняяженщинакоторуюуженелюбятноскладнаяикрепкаяименнотакаякакнадотаковыженщинывовсякомвозрастееслионилюбимыонабылаудивительнаяеетелокакиегособственноевсегдадумалозанеетолькоподругомуоновынашивалодетейиливходиловпередилеовкаждуюкомнатучтобынеуловимоизменитьтамсамыйвоздухподстатьнастроениюмужаказалосьонаникогданезадумываетсянадолгомысльтотчаспередаваласьотееголовыплечампальцамипретворяласьвдействиетакнезаметноиестественночтолеонесмогбыдаинехотелизобразитьэтокакимилибочертежамиэтамашинасказалаонанаконецненужнаонанамдаотозвалсяонноиногдан